#Trabalho de Conclusão de Curso

##Aplicação de modelos de processamento de linguagem natural em um catálogo de vulnerabilidades cibernéticas

###Identificação

**Aluno:**  José Caetano Beuker

**Curso:**  MBA em Data Science e Analytics

**IE:** Escola Superior de Agricultura Luiz de Queiroz - Universidade de São Paulo

### Importação de bibliotecas utilizadas

In [ ]:
# Turn off warnings for a cleaner look
import warnings 
warnings.simplefilter('ignore')

# Para a manipulação de dataframes e análise de dados
import pandas as pd

# Biblioteca para trabalhar com arrays grandes e multidimensionais e também matrizes
import numpy as np

# Para visualizações
import seaborn as sns

# Para visualização
import matplotlib.pyplot as plt

# Natural Language Toolkit - para PLN
import nltk 

# Dividi o texto em tokens (palavras individuais)
from nltk.tokenize import word_tokenize 

# Para remoção de stop words
from nltk.corpus import stopwords 

# Atualiza a lista de stop words
#nltk.download('stopwords')

# Cria objeto para remover stop words em Inglês
stop_words_en = stopwords.words('english')

# Para regex - regular expression
import re

# Fazer o stemming das palavras (redução de palavras ao seu radical)
from nltk.stem import PorterStemmer
porter = PorterStemmer()

# Fazer a lematização de palavras (redução de palavras à sua forma base)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Contagem de palavras
from sklearn.feature_extraction.text import CountVectorizer

# Cria matriz de TF-FDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Para nuvem de palavras
from wordcloud import WordCloud

# Para PLN
import spacy

# Carregar o modelo
we_nlp = spacy.load("en_core_web_sm")

import scipy.cluster.hierarchy as sch

### Funções utilizadas

In [ ]:
# Obtem a media dos embeddings do texto
def media_embedding(texto):
    documento = we_nlp(texto)
    return documento.vector

# Função para tokenizar palavras
def tokeniza_palavras_coluna (dataframe, coluna):
    palavras = ' '.join([word for word in dataframe[coluna]])
    # Realiza a tokenização
    tokens = word_tokenize(palavras)
    return tokens

# Função para preparar exibição de gráfico de ocorrências de palavras em uma coluna de um DataFrame
def prepara_para_grafico (dataframe, coluna):
    # Realiza a tokenização
    tokens = tokeniza_palavras_coluna(dataframe, coluna)
    # Obtém a frequência de ocorrências do token
    frequencia = nltk.FreqDist(tokens)
    pd_frequencia = pd.DataFrame({"token": list(frequencia.keys()),
                                 "frequencia": list(frequencia.values())})
    return pd_frequencia

# Função para exibir gráfico de Pareto de ocorrências de palavras em uma coluna de um DataFrame
def exibe_pareto (dados, coluna, quantidade_colunas, cor, titulo):
    plt.figure(figsize=(9, 5))
    x = sns.barplot(data = dados.nlargest(columns = coluna, n = quantidade_colunas), x = "token", y = "frequencia", color = cor)
    x.set(ylabel = "Quantidade", xlabel = "Tokens", title = titulo)
    plt.show()

# Objeto para manter apenas caracteres alfanuméricos
# mantem_apenas_alfanumerico = re.compile('[^0-9a-z #+_]')
mantem_apenas_alfanumerico = re.compile('[^a-z #+_]')

# Criar objeto para remover caracteres especiais
remove_caracteres_especiais = re.compile('[/(){}\[\]\|@,;]')

# Função para limpar os textos das colunas do dataframe
def limpa_texto(texto):
    # Converte o texto para letras minúsculas
    texto = texto.lower()

    # Subsitui caracteres por espaços 
    texto = remove_caracteres_especiais.sub(' ', texto)

    # Remote caracteres que não forem alfanuméricos
    texto = mantem_apenas_alfanumerico.sub('', texto)
    
    return texto 

# Função para remover stop words
def remove_stop_words(word_list, stop_word_list):
    tokens = word_tokenize(word_list)
    lista_sem_stop_words = ' '.join([word for word in tokens 
                                     if word not in stop_word_list])
    return lista_sem_stop_words

# Função de Stemming
def stemming (texto):
    stemmings = [porter.stem(word) for word in word_tokenize(texto)]
    return ' '.join(stemmings)

# Função de Lematização
def lematizacao (texto):
    lematizados = []
    for word in texto:
        tokens = nltk.word_tokenize(word)
        lematizado = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
        lematizados.append(lematizado)
    return lematizados  


## Coleta de dados

In [ ]:
# Coleta dados
catalogoVulnerabilidades_bruto = pd.read_csv('https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv', header=0, sep=',')
#catalogoVulnerabilidades = pd.read_csv('dados/known_exploited_vulnerabilities.csv', header=0, sep=',')

# Mantém o DataFrame obtido originalmente sem alterações
catalogoVulnerabilidades = catalogoVulnerabilidades_bruto.copy()

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID',
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [ ]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

In [ ]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['vendorProject', 'product', 'notes'], inplace = True)

In [ ]:
# Verificar alguns dados
catalogoVulnerabilidades.head(5)

In [ ]:
# Transformar tipos de dados para datas
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)

##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')

##Transforma colunas em string
catalogoVulnerabilidades[['shortDescription', 
                          'vulnerabilityName',
                          'requiredAction']] = catalogoVulnerabilidades[['shortDescription', 
                                                                         'vulnerabilityName', 
                                                                         'requiredAction']].astype('string')

In [ ]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

In [ ]:
# Vulnerabilidade mais antiga catalogada
mais_antiga = catalogoVulnerabilidades.sort_values(by='dateAdded')
mais_antiga = mais_antiga.iloc[0]
mais_antiga

In [ ]:
# Vulnerabilidade mais recente catalogada
mais_recente = catalogoVulnerabilidades.sort_values(by='dateAdded', ascending=False)
mais_recente = mais_recente.iloc[0]
mais_recente

In [ ]:
# # Copia apenas a coluna de interesse
# Descricao_curta = catalogoVulnerabilidades['shortDescription']

In [ ]:
# # Tansforma em dataframe
# Descricao_curta = pd.DataFrame(Descricao_curta)

In [ ]:
# Descricao_curta.info()

In [ ]:
# Descricao_curta.head(5)

### Analisando a coluna shortDescription

In [ ]:
# DataFrame com pontos e caracteres especiais
pd_frequencia_dados_brutos = prepara_para_grafico(catalogoVulnerabilidades, 'shortDescription')

pd_frequencia_dados_brutos.nlargest(columns = "frequencia", n = 10).sort_values(by=['frequencia'], ascending=False)

In [ ]:
# Exibe gráfico de pareto contendo caracteres especiais e stop words
exibe_pareto(pd_frequencia_dados_brutos, 'frequencia', 10, 'purple',  'Coluna shortDescription contendo caracteres especiais e stop words')

### Normalizando as palavras

In [ ]:
catalogoVulnerabilidades_limpo = catalogoVulnerabilidades.copy()

In [ ]:
# # Limpa a coluna 
# df_descricao_curta_coluna_limpa = pd.DataFrame(Descricao_curta)
# df_descricao_curta_coluna_limpa['shortDescription'] = Descricao_curta['shortDescription'].apply(lambda x: limpa_texto(str(x)).lower())
# df_descricao_curta_coluna_limpa['shortDescription']

In [ ]:
# Limpa a coluna
catalogoVulnerabilidades_limpo['shortDescription'] = catalogoVulnerabilidades_limpo['shortDescription'].apply(lambda x: limpa_texto(str(x)).lower())
catalogoVulnerabilidades_limpo['shortDescription']

In [ ]:
# Tokeniza DataFrame limpo
tokens_coluna_limpa = tokeniza_palavras_coluna(catalogoVulnerabilidades_limpo, 'shortDescription')

In [ ]:
# Carrega lista de stop words do inglês e acrescenta algumas palavras
sw_vulnerabilidade_ciberneticas = ["could", "couldn't", "vulnerability", "would", "wouldn't"] # Ver quais stop words adicionar
sw_en = list(set(stopwords.words('english')))
sw_en_plus = sw_en + sw_vulnerabilidade_ciberneticas



In [ ]:
# # Remove as stop words
# df_descricao_curta_sem_stop_words = pd.DataFrame(df_descricao_curta_coluna_limpa)
# df_descricao_curta_sem_stop_words['shortDescription'] = df_descricao_curta_sem_stop_words['shortDescription'].apply(lambda x: remove_stop_words(x, sw_en_plus))
# df_descricao_curta_sem_stop_words['shortDescription']

In [ ]:
# Remove as stop words
# df_descricao_curta_sem_stop_words = pd.DataFrame(df_descricao_curta_coluna_limpa)
catalogoVulnerabilidades_limpo['shortDescription'] = catalogoVulnerabilidades_limpo['shortDescription'].apply(lambda x: remove_stop_words(x, sw_en_plus))
catalogoVulnerabilidades_limpo['shortDescription']

In [ ]:
# Cria DataFrame sem stop words
pd_frequencia_sem_stop_words = prepara_para_grafico(catalogoVulnerabilidades_limpo, 'shortDescription')

In [ ]:
# Exibe gráfico de pareto sem caracteres especiais e sem stop words
exibe_pareto(pd_frequencia_sem_stop_words, 'frequencia', 10, 'blue',  'Coluna shortDescription sem stop words e caracteres especiais')

### Stemming

In [ ]:
catalogoVulnerabilidades_stemming = catalogoVulnerabilidades_limpo.copy()

In [ ]:
# Aplicando Stemming
catalogoVulnerabilidades_stemming['shortDescription'] = catalogoVulnerabilidades_stemming['shortDescription'].apply(lambda x: stemming(str(x)))
catalogoVulnerabilidades_stemming['shortDescription']

In [ ]:
# Prepara para exibição de gráfico com stemming
df_descricao_curta_stemming = prepara_para_grafico(catalogoVulnerabilidades_stemming, 'shortDescription')

In [ ]:
# Gráfico de Pareto depois do stemming
exibe_pareto(df_descricao_curta_stemming, 'frequencia', 10, 'yellow',  'Coluna shortDescription Stemming')

### Lemmatization

In [ ]:
catalogoVulnerabilidades_lemmatization = catalogoVulnerabilidades_limpo.copy()

In [ ]:
# Aplicando Lemmatization
catalogoVulnerabilidades_lemmatization['shortDescription'] = lematizacao(catalogoVulnerabilidades_lemmatization['shortDescription'])

catalogoVulnerabilidades_lemmatization['shortDescription']

In [ ]:
# Prepara para exibição de gráfico com lemmatization
df_frequencia_lemmatization = prepara_para_grafico(catalogoVulnerabilidades_lemmatization, 'shortDescription')

In [ ]:
# Palavras que mais ocorreram após a lemmatização
df_frequencia_lemmatization.nlargest(columns = "frequencia", n = len(df_frequencia_lemmatization))

In [ ]:
# Palavras que menos ocorreram após a lemmatização
df_frequencia_lemmatization.nsmallest(columns = "frequencia", n = len(df_frequencia_lemmatization)).sort_values(by=['frequencia'])

In [ ]:
# Tokens com mais de 100 ocorrências

df_frequencia_lemmatization_qtde = df_frequencia_lemmatization[(df_frequencia_lemmatization.frequencia >= 100)].sort_values(by = ['frequencia'], ascending=False)

df_frequencia_lemmatization_qtde.head(10)

In [ ]:
# Gráfico de Pareto depois do Lemmatization
exibe_pareto(df_frequencia_lemmatization_qtde, 'frequencia', 10, 'green', 'Coluna shortDescription após lemmatization')

### Bag of words

In [ ]:
catalogoVulnerabilidades_bow = catalogoVulnerabilidades_limpo.copy()

In [ ]:
# Vetorizar coluna e criar a bag of words (sacola de palavras)
vetorizar = CountVectorizer(lowercase = False, max_features=2450)
bow = vetorizar.fit_transform(catalogoVulnerabilidades_bow['shortDescription'])


print(bow.shape)

In [ ]:
# Tranforma matriz esparsa em Dataframe
matriz_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

matriz_bow.head()

### Wordcloud

In [ ]:
# Wordcloud
coluna_em_analise_all_words = ' '.join([word for word in catalogoVulnerabilidades_limpo['shortDescription']])

# Quantidade de palavras
print(len(coluna_em_analise_all_words))

In [ ]:
# Gerar wordcloud

## https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

coluna_em_analise_wc = WordCloud(width= 800, height= 500, max_font_size = 110, collocations=False).generate(coluna_em_analise_all_words)

In [ ]:
# Plotar wordcloud
plt.figure(figsize=(12, 6))
plt.imshow(coluna_em_analise_wc, interpolation='bilinear') #ver outras interpolações
plt.axis("off")
plt.show()

### Word Embeddings

In [ ]:
catalogoVulnerabilidades_we = catalogoVulnerabilidades_limpo.copy()

In [ ]:
# we_df = pd.DataFrame(df_descricao_curta_lemmatization)

catalogoVulnerabilidades_we['embedding'] = catalogoVulnerabilidades_we['shortDescription'].apply(media_embedding)

catalogoVulnerabilidades_we[['cveID','shortDescription', 'embedding']]

In [ ]:
catalogoVulnerabilidades_we.info()

### TF-IDF

In [ ]:
# Inicialize o vetorizador TF-IDF
vetorizador = TfidfVectorizer(lowercase=False, max_features=300)

In [ ]:
# Aplique o vetorizador aos dados
matriz_tfidf = vetorizador.fit_transform(catalogoVulnerabilidades_limpo["shortDescription"])

In [ ]:
tfidf_df_descricao_curta = pd.DataFrame(matriz_tfidf.toarray(), columns=vetorizador.get_feature_names_out())

tfidf_df_descricao_curta.head(10)

### Ngrams

In [ ]:
# Inicialize o vetorizador Ngrams ( Considerando os bigramas)
vetorizador = TfidfVectorizer(lowercase=False, max_features=300, ngram_range=(2,2))

In [ ]:
# Aplique o vetorizador aos dados
matriz_ngrams = vetorizador.fit_transform(catalogoVulnerabilidades_limpo["shortDescription"])

In [ ]:
ngrams_df_descricao_curta = pd.DataFrame(matriz_ngrams.toarray(), columns=vetorizador.get_feature_names_out())

ngrams_df_descricao_curta.head(10)

## Análise Exploratória dos Dados
### Analisando a coluna shortDescription

## Modelagem
### Agrupamento

In [ ]:
catalogoVulnerabilidades_we.info()

In [ ]:
catalogoVulnerabilidades_cve = catalogoVulnerabilidades_we[['cveID', 'embedding']]

print(catalogoVulnerabilidades_cve)

### Análise de sentimento